In [13]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2
from config import db_password,g_key
from config import resolution,lat_max,lat_min,lng_max,lng_min
from config import t_sat,flow_rate,project_span
import json
from shapely.geometry import Point,shape
import os
from datetime import datetime,timedelta
from functions import Rmax_calc,recovery_calc,month_index
import scipy.optimize
from geojson import Feature, FeatureCollection, Polygon

In [14]:
# blocks information
file_path = os.path.join('..','..','Resources','blocks_info.csv')
blocks_df = pd.read_csv(file_path)
# actual gold produciton by month
file_path = os.path.join('..','..','Resources','act_monthly_oz.csv')
act_oz_df = pd.read_csv(file_path)

In [15]:
# Convert to datetime format
blocks_df['stack_finish'] = pd.to_datetime(blocks_df['stack_finish'])
blocks_df['leach_start'] = pd.to_datetime(blocks_df['leach_start'])
blocks_df['leach_end'] = pd.to_datetime(blocks_df['leach_end'])
blocks_df['leach_days'] = blocks_df['leach_end']-blocks_df['leach_start']
lat_lng_list = blocks_df[['lat','lng']].values.tolist()
# Create a dataframe with sequence as block_id, lift and lat_lng as columns
block_leach_time = pd.DataFrame(blocks_df[['lift']])
block_leach_time['lat_lng']=blocks_df[['lat','lng']].values.tolist()
block_leach_time.index.name = 'block_id'
# Create block_list for later index reference.
block_list = block_leach_time.index.tolist()
# Break dataframe to records
blocks_records = blocks_df.to_records(index=False)

In [37]:
blocks_df.iloc[0]['stack_finish']

Timestamp('2019-03-27 00:00:00')

In [46]:
pd.to_datetime(blocks_records[0]['stack_finish'])


Timestamp('2019-03-27 00:00:00')

In [40]:
date = (blocks_records[0]['stack_finish'])
year = date.astype('datetime64[Y]').astype(int) + 1970
month = date.astype('datetime64[M]').astype(int)%12+1

In [43]:
project_span

{'startdate': '2019-01-01', 'enddate': '2025-12-31'}

In [35]:
def month_index_redo(date,project_span):
    year = date.astype('datetime64[Y]').astype(int) + 1970
    month = date.astype('datetime64[M]').astype(int) % 12 + 1
    return (year-project_span[0].year)*12 + month -project_span[0].month

In [10]:
# Set up project span
projectspan = (
    datetime.strptime(project_span['startdate'],'%Y-%m-%d'),
    datetime.strptime(project_span['enddate'],'%Y-%m-%d')
    )
# Create month list of the project
startyear = projectspan[0].year
startmonth = projectspan[0].month
endyear = projectspan[1].year
endmonth = projectspan[1].month
monthlist = [datetime(m//12, m%12+1, 1) for m in range(startyear*12+startmonth-1, endyear*12+endmonth)]
# Use monthlist create recovery dataframe
rec_df = pd.DataFrame(monthlist,columns = ['months'])

In [45]:
blocks_df

,lat,lng,lift,cell,stack_finish,leach_start,leach_end,leach_days,tons_per_block,ounces_per_block,rmax
block_id,,,,,,,,,,,
0,65.012000,-147.351429,1,1,2019-03-27,2019-03-30,2019-07-28,120 days,1994.483436,16.809887,0.706002
1,65.012000,-147.350612,1,1,2019-03-27,2019-03-30,2019-07-28,120 days,1994.483436,16.809887,0.706002
2,65.012265,-147.353061,1,1,2019-03-27,2019-03-30,2019-07-28,120 days,1994.483436,16.809887,0.706002
3,65.012265,-147.352245,1,1,2019-03-27,2019-03-30,2019-07-28,120 days,1994.483436,16.809887,0.706002
4,65.012265,-147.351429,1,1,2019-03-27,2019-03-30,2019-07-28,120 days,1994.483436,16.809887,0.706002
...,...,...,...,...,...,...,...,...,...,...,...
5144,65.021816,-147.362857,4,18,2020-09-27,2020-09-30,2021-01-28,120 days,10303.913043,140.390869,0.738674
5145,65.021816,-147.362041,4,18,2020-09-27,2020-09-30,2021-01-28,120 days,10303.913043,140.390869,0.738674
5146,65.021816,-147.361224,4,18,2020-09-27,2020-09-30,2021-01-28,120 days,10303.913043,140.390869,0.738674


In [33]:
# This block of code calculates a specific block's recovery over months.
## The function takes two block indices 
## and the k factor that is going to be used for recovery calculation.
def block_months_rec(i,ref,k,lift_diff):
    block = blocks_records[i]
    # if the two indices are the same, means this is calculating the primary leach recovery
    if i == ref:
        start_date = block['leach_start']
        end_date = block['leach_end']
    # if the two indices are different, means calculating recoveries of blocks under secondary leach
    else:
        # ref_block is the block on the very top
        ref_block = blocks_records[ref]
        # the blocks below the ref_block inherit the leaching dates from the ref_block
        # calculate leach delay
        start_date = ref_block['leach_start'] + timedelta(days = t_sat + round(lift_diff * flow_rate))
        end_date = ref_block['leach_end'] + timedelta(days = round(lift_diff * flow_rate))
    # locate the month row
    m_index = month_index_redo(start_date,projectspan)
    start_month = start_date.month
    end_month = end_date.month + (end_date.year - start_date.year) * 12
    rmax = block['rmax']
    # calcuate how much recovery that this block has achieved since the start
    rcum = sum(rec_df.iloc[0:m_index][i])
    if rcum < rmax: # if cumulative recovery has not achieved the maximum recovery
        for m in range(start_month,end_month+1):
            # calculate how many days in this month for recovery calculation
            if m == start_month:
                if m == 12:
                    days = 31 - start_date.day
                else:
                    days = (datetime(start_date.year,start_date.month+1,1)-start_date).days
            elif m == end_month:
                days = end_date.day
            elif m < 12:
                days = (datetime(start_date.year,m+1,1)-datetime(start_date.year,m,1)).days
            elif m == 12:
                days = 31
            else:
                days = (datetime(start_date.year+1,m-12+1,1)-datetime(start_date.year+1,m-12,1)).days
            # recovery calculation here.
            r = recovery_calc(rmax,rcum,k,days)
            if rcum + r > rmax:
                r = rmax - rcum
                rcum = 1
            else:
                rcum += r
            # write the recovery to the rec_df.
            rec_df.at[m_index,i] = r
            m_index += 1

In [34]:
## k factor here will be changed to a formula
k = 0.001
def blocks_recoveries(k):
    for high in block_list:
        this_block = blocks_df.iloc[high]
        rec_df[high]=0.0
        lift_diff = 0
        block_months_rec(high,high,k,lift_diff)
        high_lift = this_block['lift']
        # After this block's recovery has been calculated,
        # start calculating any blocks that are below this block.
        for low in range(0,high):
            if lat_lng_list[high]==lat_lng_list[low]:
                low_lift = blocks_df.iloc[low]['lift']
                lift_diff = high_lift-low_lift
                block_months_rec(low,high,k,lift_diff)

blocks_recoveries(k)

AttributeError: 'numpy.datetime64' object has no attribute 'month'